In [1]:
#Import Packages
import pandas as pd
import numpy as np
import os
from persiantools.jdatetime import JalaliDate as dt

In [6]:

#Inputs
exls_dir=r'/home/anvaari/Python_Works/صندوق سرمایه گذاری/سهامی/Data' #Directory where excels file exist
shakhes_name='Shakhes.xlsx' #Name of xlsx file contain shakhes value
month='05' #Which month do you want to start from ---> in this form (str) : '05' or '10'
day_range=(30,10) #Range of days in month _ For example: (30,10) it start from 30th day of month and go back (to avoid errors)

In [7]:
'''
This function get NAVs and return dictionary where its keys are years of stock fund and each value is revenue during that year. 
In mode 1 function calculate it year by year
In mode 2 fuunction calculate it every 2 years
In mode 5 fuunction calculate it every 5 years

'''
def bazdeh_calc(data,data_ebtal,mode):
    bazdeh={}
    keys=list(data.keys())
    if mode==1:
        for year in range(keys[0],keys[-1]):
            bazdeh[year]=(data_ebtal[year+1]-data[year])/data[year]*100
    elif mode==2:
        if len(data)<2: #check data have at least data of 2 year 
            return np.nan
        for year in range(keys[0],keys[-2]):
            bazdeh[year]=(data_ebtal[year+2]-data[year])/data[year]*100
    elif mode==5:
        if len(data)<6: #check data have at least data of 5 year 
            return np.nan
        else:
            bazdeh[keys[-6]]=(data_ebtal[keys[-1]]-data[keys[-6]])/data[keys[-6]]*100
            
    return bazdeh

In [8]:
names=os.listdir(exls_dir) #Get list of file in exls_dir directory
names.remove(shakhes_name) 
result=pd.DataFrame(columns=['Name','Sal Tasis']) #Creat final dataframe. now it is empty
names=['/'+x.strip('\n').strip('~$') for x in names]
shakhes=pd.read_excel(exls_dir+'/'+shakhes_name) #Read Shakhes data from its file

shakhes_data=pd.DataFrame()
for name in names: #Loop on every stock fund's name
    nav_yearly={} 
    nav_yearly_ebtal={}
    temp = pd.read_excel(exls_dir+name) #This dataframe hold data of "name" in each loop.
    afz=False #Boolean for detect Capital Increase (Afzayesh Sarmaye)
    date_afz=dt(2000,2,2) #Date of Capital Increase (Afzayesh Sarmaye)
    for index, row in temp.iterrows(): #This loop itrate over every item and try to detect if Capital Increase (Afzayesh Sarmaye) exist or not.
        try:
            if row['NAVSodoor']/temp.loc[index+1,'NAVSodoor'] <= 0.5:
                afz=True
                temp_date=temp.loc[index,'Date']
                date_afz=dt(int(temp_date[:4]),int(temp_date[5:7]),int(temp_date[-2:]))
                factor=temp.loc[index+1,'NAVSodoor']/row['NAVSodoor'] #This factor used for balance NAVs after Capital Increase (Afzayesh Sarmaye)
                factor_ebtal=temp.loc[index+1,'NAVEbtal']/row['NAVEbtal']
        except :
            pass
    date_start=temp.iloc[-1]['Date'] #Start date of activity of "name"
    year_start=int(date_start[:4])
    year_end=int(temp.iloc[0]['Date'][:4])
    for j in range(year_start,year_end+1): #in this loop we creat Dictionary contain NAV of each year in specified date.
        if int(date_start[5:7])>int(month)+1 and j==int(date_start[:4]): #Check if month of start greater than month we wanth to analyze else go to next year
            year_start+=1
            continue
        for i in range(day_range[0],day_range[1],-1):
            date_now='{}{}{}'.format(j,f'/{month}/',i)
            date_now_dt=dt(int(date_now[:4]),int(date_now[5:7]),int(date_now[-2:]))
            if j==1396 and name=='/Melli.xlsx': #Melli fund doesn't have data for month 5 of year 1396
                date_now='{}{}{}'.format(j,'/06/',i)
            nav_now=temp[temp['Date']==date_now]['NAVSodoor']
            nav_now_ebtal=temp[temp['Date']==date_now]['NAVEbtal']
            if date_now_dt>date_afz:
                nav_now=nav_now*factor
                nav_now_ebtal=nav_now_ebtal*factor_ebtal
            if len(nav_now)==0:
                continue
            else:
                nav_yearly[j]=nav_now.iloc[0]
                nav_yearly_ebtal[j]=nav_now_ebtal.iloc[0]
                break
            
    bazdeh1=bazdeh_calc(nav_yearly,nav_yearly_ebtal, 1)
    bazdeh2=bazdeh_calc(nav_yearly,nav_yearly_ebtal, 2)
    bazdeh5=bazdeh_calc(nav_yearly,nav_yearly_ebtal, 5)
    
    #Put data on final DataFrame
    for year in bazdeh1.keys(): #Put revenues on DataFrame
        result.at[names.index(name),year]=bazdeh1[year]
        
    result.at[names.index(name),'Name']=temp.loc[0]['name']
    result.at[names.index(name),'Sal Tasis']=int(date_start[:4])
    result.at[names.index(name),'NAV Sodoor']=temp.loc[0]['NAVSodoor']
    result.at[names.index(name),'Miangin Salane']=pd.Series(bazdeh1).mean()
    result.at[names.index(name),'Miangin 2 sal']=pd.Series(bazdeh2).mean()
    result.at[names.index(name),'5 sal']=pd.Series(bazdeh5).iloc[0]
    
    #Calculate total revenue of "name" 
    nav_first=temp.iloc[-1]['NAVSodoor']
    nav_last=temp.iloc[0]['NAVEbtal']
    if afz==True:
        nav_last=nav_last*factor_ebtal
    result.at[names.index(name),'Kol']=(nav_last-nav_first)/nav_first*100

<ipython-input-8-eb94d147691d>:60: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  result.at[names.index(name),'Miangin 2 sal']=pd.Series(bazdeh2).mean()


In [9]:
#Calculate revenue of 'Shakhes kol'
shakhes_yearly={}
for j in range(1389,1400):
    for i in range(day_range[0],day_range[1],-1):
        try:
            date_now_shakhes=int('{}{}{}'.format(j,month,i))
            shakhes_now=shakhes[shakhes['dateissue']==date_now_shakhes]['Value']
            shakhes_yearly[j]=shakhes_now.iloc[0]
        except:
            continue
        else:
            break
            
bazdeh_shakhes1=bazdeh_calc(shakhes_yearly, shakhes_yearly, 1)
bazdeh_shakhes2=bazdeh_calc(shakhes_yearly, shakhes_yearly, 2)
bazdeh_shakhes5=bazdeh_calc(shakhes_yearly, shakhes_yearly, 5)
    
for year in bazdeh_shakhes1.keys():
    shakhes_data.at[0,year]=bazdeh_shakhes1[year]
shakhes_data.at[0,'Miangin Salane']=pd.Series(bazdeh_shakhes1).mean()
shakhes_data.at[0,'Miangin 2 sal']=pd.Series(bazdeh_shakhes2).mean()
shakhes_data.at[0,'5 sal']=pd.Series(bazdeh_shakhes5).iloc[0]
shakhes_data.at[0,'Kol']=(shakhes.iloc[0]['Value']-shakhes.iloc[-1]['Value'])/shakhes.iloc[-1]['Value']*100

In [10]:
#Calculate mean score (if revenue of stock fund greater than mean of others in that year score +1) and Shakhes Score (if revenue of investment fund greater than revenue of 'Shakhes' in that year score +1) in each year
year_info=pd.DataFrame(result.loc[:,[i for i in range(1389,1399)]].describe()) #Summary of result DataFrame
for index,row in result.iterrows():
    score_m=0
    score_sh=0
    for i in range(1389,1400):
        try:
            if row[i]>year_info.at['mean',i]:
                score_m+=1
            if row[i]>shakhes_data.at[0,i]:
                score_sh+=1
        except:
            continue

    result.at[index,'Score Miangin']=score_m
    result.at[index,'Score Shakhes']=score_sh
    
col=[i for i in range(1389,1399)]
col.append('Score Miangin')
col.append('Score Shakhes')
year_info=pd.DataFrame(result.loc[:,col].describe())

In [25]:
#Sort result in different ways
best_kol=pd.DataFrame(result.sort_values(by=['Kol'],ascending=False)).reset_index(drop=True)
best_5=pd.DataFrame(result.sort_values(by=['5 sal'],ascending=False)).reset_index(drop=True)
best_mian=pd.DataFrame(result.sort_values(by=['Score Miangin'],ascending=False)).reset_index(drop=True)
best_shakhes=pd.DataFrame(result.sort_values(by=['Score Shakhes'],ascending=False)).reset_index(drop=True)

#Select Best stock fund and Print it
#Change condition with respect to your expectations
print("صندوق(های) منتخب با توجه به شرایط اعلامی :\n")
for name in best_kol.head(10)['Name'].values:
    if name in best_5.head(5)['Name'].values and name in best_mian.head(12)['Name'].values and name in best_shakhes.head(10)['Name'].values:
        print(name)

صندوق(های) منتخب با توجه به شرایط اعلامی :

ارزش کاوان آینده


In [13]:
result.head(10)

,Name,Sal Tasis,1389,1390,1391,1392,1393,1394,1395,1396,1397,1398,NAV Sodoor,Miangin Salane,Miangin 2 sal,5 sal,Kol,Score Miangin,Score Shakhes
0,صندوق سرمايه گذاري مشترک کارگزاري حافظ,1388,25.936521,12.103910,154.369457,29.164962,-3.874339,26.979497,-9.054583,48.250922,91.294215,526.365168,15995793.0,90.153573,207.921272,2057.536772,12328.610588,6.0,5.0
1,صندوق سرمایه‌گذاری مشترک پیشتاز,1388,52.145184,-0.611411,110.362800,28.470489,-1.098189,26.461411,7.379962,67.566104,93.607831,385.328878,250086337.0,76.961306,176.827945,2136.975485,15113.189582,6.0,7.0
2,مشترک گنجینه ارمغان الماس,1395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.320526,82.193067,399.112710,18194188.0,185.875434,523.272394,NaN,1707.767400,1.0,1.0
3,صندوق سرمایه‌گذاری آسمان آرمانی سهام,1392,NaN,NaN,NaN,NaN,2.418189,17.708861,12.892615,30.947525,97.174232,533.606644,175384.0,115.791344,286.577447,2188.924051,1643.190000,3.0,3.0
4,مشترک عقیق,1389,NaN,4.975814,211.553005,30.140707,-13.097491,29.662169,-11.540225,61.129090,87.896355,446.397710,86510764.0,94.124126,222.425994,1911.249649,8454.120497,5.0,4.0
5,صندوق سرمایه‌گذاری گنجینه رفاه,1389,36.874068,-21.256666,209.301531,-24.031482,-16.903621,22.861529,-1.004127,75.664085,134.943449,596.919770,89836527.0,101.336854,249.178901,3594.207862,8823.583800,4.0,5.0
6,قابل معامله سرو سودمند مدبران,1398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,527.342117,68491.0,527.342117,NaN,NaN,567.566241,1.0,0.0
7,مشترک توسعه ملی,1392,NaN,NaN,NaN,NaN,-13.365527,12.527131,6.004089,68.624736,137.325332,497.210050,25278993.0,118.054302,348.821458,2925.057131,2409.592308,3.0,3.0
8,همیان سپهر,1393,NaN,NaN,NaN,NaN,-22.712718,14.473589,7.452565,32.023318,64.410490,576.985593,14842564.0,112.105473,240.600980,1782.948761,1329.987871,2.0,2.0
9,قابل معامله هستی بخش آگاه,1394,NaN,NaN,NaN,NaN,NaN,NaN,-3.373952,69.948224,100.400634,525.617848,300069.0,173.148189,494.226299,NaN,2737.320255,3.0,2.0


In [14]:
year_info

,1389,1390,1391,1392,1393,1394,1395,1396,1397,1398,Score Miangin,Score Shakhes
count,14.000000,22.000000,25.000000,26.000000,33.000000,38.000000,40.000000,43.000000,44.000000,45.000000,45.000000,45.000000
mean,40.421070,-0.252861,147.301538,17.581245,-9.342710,25.323462,0.269122,54.623234,100.218683,505.321578,3.400000,3.111111
std,16.584612,15.990839,49.664187,14.632082,8.328836,13.669716,11.051385,25.447541,25.587975,102.662371,1.498484,1.885618
min,11.222075,-28.296421,72.580437,-24.031482,-27.263330,-0.455937,-18.126688,-1.345615,53.835444,198.962453,1.000000,0.000000
25%,29.282919,-11.190242,110.362800,8.164714,-14.709901,17.372402,-7.605973,36.720796,86.720685,436.245849,2.000000,2.000000
50%,39.446476,-1.734949,151.268500,19.477751,-10.670032,22.799115,-1.097576,56.496235,94.030045,520.317673,3.000000,3.000000
75%,49.253662,6.275604,200.133218,28.421616,-1.873159,28.758649,5.013733,72.806155,115.197390,565.689421,5.000000,4.000000
max,81.477329,28.151634,227.831304,42.336581,7.249600,57.940130,44.461464,119.672410,162.902176,803.143580,6.000000,7.000000


In [17]:
best_kol.head(10)

,Name,Sal Tasis,1389,1390,1391,1392,1393,1394,1395,1396,1397,1398,NAV Sodoor,Miangin Salane,Miangin 2 sal,5 sal,Kol,Score Miangin,Score Shakhes
0,مشترک بورسیران,1388,49.671314,20.109578,227.831304,17.138885,-17.812076,18.466918,11.297676,22.021442,150.320821,583.154381,331487041.0,108.220024,287.227929,2817.737288,26391.665621,6.0,6.0
1,مشترک آگاه,1388,39.319986,6.542081,200.133218,27.193306,-11.870021,26.219522,-11.604503,80.342177,106.022826,450.945120,377877380.0,91.324371,221.042508,2324.254308,25011.198368,6.0,6.0
2,صندوق سرمایه‌گذاری مشترک پیشتاز,1388,52.145184,-0.611411,110.362800,28.470489,-1.098189,26.461411,7.379962,67.566104,93.607831,385.328878,250086337.0,76.961306,176.827945,2136.975485,15113.189582,6.0,7.0
3,مشترك كارگزاري بانك ملي ايران,1388,30.555667,5.052568,153.880478,17.928053,-14.057418,33.381242,-7.606637,96.464513,99.120752,436.245849,353618519.0,85.096507,202.109992,2633.727522,15032.780751,5.0,5.0
4,مشترک پویا,1388,39.572965,-10.093561,164.154930,33.249219,-5.748794,15.155503,-8.525158,47.256633,84.828566,496.563106,222564630.0,85.641341,188.945841,1707.714415,14866.611133,3.0,3.0
5,یکم اکسیر فارابی,1388,48.000703,-7.104070,200.934769,10.681681,-27.263330,34.251581,-12.957189,80.518040,127.215233,401.819258,146753442.0,85.609668,210.250455,2458.628795,14482.508360,5.0,4.0
6,ارزش کاوان آینده,1388,51.607166,-28.296421,72.580437,4.777999,-0.529139,49.804735,-0.932531,35.118258,162.902176,803.143580,171358981.0,115.017626,314.713018,4974.870410,13958.029682,5.0,5.0
7,صندوق سرمايه گذاري مشترک کارگزاري حافظ,1388,25.936521,12.103910,154.369457,29.164962,-3.874339,26.979497,-9.054583,48.250922,91.294215,526.365168,15995793.0,90.153573,207.921272,2057.536772,12328.610588,6.0,5.0
8,مشترک نقش جهان,1389,43.219900,-6.570002,93.718119,3.522886,-4.252802,57.940130,-0.532371,30.916803,144.121116,746.343966,1183224.0,110.842774,286.784320,4378.474708,11817.546402,5.0,4.0
9,صندوق سرمایه‌گذاری فیروزه موفقیت,1389,27.433428,24.310155,84.544019,42.336581,-13.051751,38.976062,14.382796,40.691030,126.304776,384.911530,94791862.0,77.083863,195.420263,2465.193653,9335.210300,5.0,5.0


In [19]:
best_5.head(10)

,Name,Sal Tasis,1389,1390,1391,1392,1393,1394,1395,1396,1397,1398,NAV Sodoor,Miangin Salane,Miangin 2 sal,5 sal,Kol,Score Miangin,Score Shakhes
0,ارزش کاوان آینده,1388,51.607166,-28.296421,72.580437,4.777999,-0.529139,49.804735,-0.932531,35.118258,162.902176,803.143580,171358981.0,115.017626,314.713018,4974.870410,13958.029682,5.0,5.0
1,مشترک نقش جهان,1389,43.219900,-6.570002,93.718119,3.522886,-4.252802,57.940130,-0.532371,30.916803,144.121116,746.343966,1183224.0,110.842774,286.784320,4378.474708,11817.546402,5.0,4.0
2,صندوق سرمایه‌گذاری گنجینه رفاه,1389,36.874068,-21.256666,209.301531,-24.031482,-16.903621,22.861529,-1.004127,75.664085,134.943449,596.919770,89836527.0,101.336854,249.178901,3594.207862,8823.583800,4.0,5.0
3,صندوق سرمایه‌گذاری بانک توسعه تعاون,1391,NaN,NaN,207.564826,-10.941042,-18.869614,20.721304,-1.563931,78.492263,130.603702,593.091316,74014191.0,124.887353,298.303685,3486.342550,7250.594100,4.0,5.0
4,صندوق سرمایه‌گذاری مشترک مبین سرمایه,1394,NaN,NaN,NaN,NaN,NaN,55.856100,-4.086045,81.026472,88.488850,510.548662,31675150.0,146.366808,360.243804,3198.445800,3047.743700,3.0,2.0
5,مشترک توسعه ملی,1392,NaN,NaN,NaN,NaN,-13.365527,12.527131,6.004089,68.624736,137.325332,497.210050,25278993.0,118.054302,348.821458,2925.057131,2409.592308,3.0,3.0
6,مشترک بورسیران,1388,49.671314,20.109578,227.831304,17.138885,-17.812076,18.466918,11.297676,22.021442,150.320821,583.154381,331487041.0,108.220024,287.227929,2817.737288,26391.665621,6.0,6.0
7,مشترک بانک خاورمیانه,1392,NaN,NaN,NaN,NaN,-10.670032,21.230378,2.964962,26.888102,144.771261,581.989136,25539019.0,127.862301,375.421010,2701.594028,2437.047722,3.0,4.0
8,مشترك كارگزاري بانك ملي ايران,1388,30.555667,5.052568,153.880478,17.928053,-14.057418,33.381242,-7.606637,96.464513,99.120752,436.245849,353618519.0,85.096507,202.109992,2633.727522,15032.780751,5.0,5.0
9,مشترك خوارزمي,1390,NaN,-6.002276,134.288001,14.501971,-1.807131,9.609032,6.067566,96.157416,77.016546,528.793675,67070631.0,95.402755,215.814370,2582.454689,6562.778704,4.0,3.0


In [20]:
best_mian.head(10)

,Name,Sal Tasis,1389,1390,1391,1392,1393,1394,1395,1396,1397,1398,NAV Sodoor,Miangin Salane,Miangin 2 sal,5 sal,Kol,Score Miangin,Score Shakhes
0,صندوق سرمايه گذاري مشترک کارگزاري حافظ,1388,25.936521,12.103910,154.369457,29.164962,-3.874339,26.979497,-9.054583,48.250922,91.294215,526.365168,15995793.0,90.153573,207.921272,2057.536772,12328.610588,6.0,5.0
1,صندوق سرمایه‌گذاری مشترک پیشتاز,1388,52.145184,-0.611411,110.362800,28.470489,-1.098189,26.461411,7.379962,67.566104,93.607831,385.328878,250086337.0,76.961306,176.827945,2136.975485,15113.189582,6.0,7.0
2,مشترک بورسیران,1388,49.671314,20.109578,227.831304,17.138885,-17.812076,18.466918,11.297676,22.021442,150.320821,583.154381,331487041.0,108.220024,287.227929,2817.737288,26391.665621,6.0,6.0
3,مشترک آگاه,1388,39.319986,6.542081,200.133218,27.193306,-11.870021,26.219522,-11.604503,80.342177,106.022826,450.945120,377877380.0,91.324371,221.042508,2324.254308,25011.198368,6.0,6.0
4,توسعه صادرات,1390,NaN,28.151634,161.166859,7.325725,-5.766843,53.456752,-18.126688,19.164550,98.829968,588.839329,77906511.0,103.671254,241.320444,2100.459132,7646.648578,5.0,6.0
5,مشترک نقش جهان,1389,43.219900,-6.570002,93.718119,3.522886,-4.252802,57.940130,-0.532371,30.916803,144.121116,746.343966,1183224.0,110.842774,286.784320,4378.474708,11817.546402,5.0,4.0
6,مشترك كارگزاري بانك ملي ايران,1388,30.555667,5.052568,153.880478,17.928053,-14.057418,33.381242,-7.606637,96.464513,99.120752,436.245849,353618519.0,85.096507,202.109992,2633.727522,15032.780751,5.0,5.0
7,ارزش کاوان آینده,1388,51.607166,-28.296421,72.580437,4.777999,-0.529139,49.804735,-0.932531,35.118258,162.902176,803.143580,171358981.0,115.017626,314.713018,4974.870410,13958.029682,5.0,5.0
8,صندوق سرمایه‌گذاری فیروزه موفقیت,1389,27.433428,24.310155,84.544019,42.336581,-13.051751,38.976062,14.382796,40.691030,126.304776,384.911530,94791862.0,77.083863,195.420263,2465.193653,9335.210300,5.0,5.0
9,یکم اکسیر فارابی,1388,48.000703,-7.104070,200.934769,10.681681,-27.263330,34.251581,-12.957189,80.518040,127.215233,401.819258,146753442.0,85.609668,210.250455,2458.628795,14482.508360,5.0,4.0


In [21]:
best_shakhes.head(10)

,Name,Sal Tasis,1389,1390,1391,1392,1393,1394,1395,1396,1397,1398,NAV Sodoor,Miangin Salane,Miangin 2 sal,5 sal,Kol,Score Miangin,Score Shakhes
0,صندوق سرمایه‌گذاری مشترک پیشتاز,1388,52.145184,-0.611411,110.362800,28.470489,-1.098189,26.461411,7.379962,67.566104,93.607831,385.328878,250086337.0,76.961306,176.827945,2136.975485,15113.189582,6.0,7.0
1,صندوق سرمایه‌گذاری مشترک پیشرو,1390,NaN,5.476174,114.280460,26.945706,-0.460356,22.736701,4.094906,67.047083,98.843054,359.380122,591481.0,77.593761,190.195237,1944.494124,5724.231818,5.0,6.0
2,مشترک بورسیران,1388,49.671314,20.109578,227.831304,17.138885,-17.812076,18.466918,11.297676,22.021442,150.320821,583.154381,331487041.0,108.220024,287.227929,2817.737288,26391.665621,6.0,6.0
3,توسعه صادرات,1390,NaN,28.151634,161.166859,7.325725,-5.766843,53.456752,-18.126688,19.164550,98.829968,588.839329,77906511.0,103.671254,241.320444,2100.459132,7646.648578,5.0,6.0
4,مشترک آگاه,1388,39.319986,6.542081,200.133218,27.193306,-11.870021,26.219522,-11.604503,80.342177,106.022826,450.945120,377877380.0,91.324371,221.042508,2324.254308,25011.198368,6.0,6.0
5,صندوق سرمايه گذاري مشترک کارگزاري حافظ,1388,25.936521,12.103910,154.369457,29.164962,-3.874339,26.979497,-9.054583,48.250922,91.294215,526.365168,15995793.0,90.153573,207.921272,2057.536772,12328.610588,6.0,5.0
6,صندوق سرمایه‌گذاری گنجینه رفاه,1389,36.874068,-21.256666,209.301531,-24.031482,-16.903621,22.861529,-1.004127,75.664085,134.943449,596.919770,89836527.0,101.336854,249.178901,3594.207862,8823.583800,4.0,5.0
7,صندوق سرمایه‌گذاری بانک توسعه تعاون,1391,NaN,NaN,207.564826,-10.941042,-18.869614,20.721304,-1.563931,78.492263,130.603702,593.091316,74014191.0,124.887353,298.303685,3486.342550,7250.594100,4.0,5.0
8,صندوق سرمایه‌گذاری فیروزه موفقیت,1389,27.433428,24.310155,84.544019,42.336581,-13.051751,38.976062,14.382796,40.691030,126.304776,384.911530,94791862.0,77.083863,195.420263,2465.193653,9335.210300,5.0,5.0
9,ارزش کاوان آینده,1388,51.607166,-28.296421,72.580437,4.777999,-0.529139,49.804735,-0.932531,35.118258,162.902176,803.143580,171358981.0,115.017626,314.713018,4974.870410,13958.029682,5.0,5.0


In [ ]:
#Export Result
result.to_excel('Final.xlsx')
year_info.to_excel('Final_Summary.xlsx')